<a href="https://colab.research.google.com/github/dsahoo17/GestureRecognition/blob/master/Flair_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus, ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings,DocumentLSTMEmbeddings, BertEmbeddings, StackedEmbeddings, TokenEmbeddings
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from typing import List
from flair.data import Sentence

In [2]:
with open('flair/train.txt', 'r') as train_txt: 
    print(train_txt.read())

show O
me O
the O
flights O
to O
love I-location_detail
field I-location_detail

show O
me O
all O
flights O
from O
boston I-location_detail
to O
denver I-location_detail

philadelphia I-location_detail
to O
san I-location_detail
francisco I-location_detail
monday I-depart

from O
san O
francisco O
on O
tuesday I-depart

show O
me O
all O
flights O
from O
san I-location_detail
francisco I-location_detail
to O
washington I-location_detail
dc I-location_detail
area O

sure O
i O
want O
to O
go O
from O
philadelphia I-location_detail
to O
dallas I-location_detail

show O
me O
all O
the O
available O
flights O
from O
baltimore I-location_detail
to O
dallas I-location_detail
with O
economy I-aircraft_detail
fares O

show O
me O
the O
flights O
from O
philadelphia I-location_detail
to O
atlanta I-location_detail

show O
me O
the O
latest I-flight_detail
dinner I-aircraft_detail
flight O
from O
baltimore I-location_detail
to O
oakland I-location_detail




In [3]:
import os

In [4]:

data_folder = "./flair"
model_dir = "./"
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus(data_folder, columns,
                                      train_file='train.txt',
                                      #test_file='test.txt',
                                      # dev_file='dev.txt'
                                      )

2020-07-18 16:57:25,312 Reading data from flair
2020-07-18 16:57:25,315 Train: flair/train.txt
2020-07-18 16:57:25,318 Dev: None
2020-07-18 16:57:25,319 Test: None


In [5]:
tag_type = 'ner'

#on utilise la deuxième colonne ici correspondant aux labels pour la ner

tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
#on récupère la liste des labels


embedding_types: List[TokenEmbeddings] = [

            WordEmbeddings('fr'),
            FlairEmbeddings('fr-forward'),
            FlairEmbeddings('fr-backward'),
        ]

In [6]:
#permet de combiner les embeddings
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                                embeddings=embeddings,
                                                tag_dictionary=tag_dictionary,
                                                tag_type=tag_type,
                                                use_crf=True)
#regarder tous les paramètres

In [7]:
#On crée un objet pour train avec flair. Ici on utilise seulement des données de train
trainer = ModelTrainer(tagger, corpus)

#On lance le training
trainer.train(model_dir,learning_rate=0.1,mini_batch_size=32, max_epochs=10,embeddings_storage_mode='cpu',)


2020-07-18 16:57:37,461 ----------------------------------------------------------------------------------------------------
2020-07-18 16:57:37,463 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('fr')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2348, out_features=2348, bias=True)
  (rnn): LSTM(2348, 256, batch_first=True, bidire

2020-07-18 17:04:20,545 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8182,
 'dev_score_history': [0.375,
  0.75,
  0.75,
  0.75,
  0.75,
  0.875,
  0.75,
  0.875,
  0.75,
  0.875],
 'train_loss_history': [22.2252254486084,
  17.393964767456055,
  13.377035140991211,
  10.910445213317871,
  9.620040893554688,
  10.57906436920166,
  8.506880760192871,
  9.52314567565918,
  8.138160705566406,
  9.151495933532715],
 'dev_loss_history': [18.136981964111328,
  12.955446243286133,
  9.529614448547363,
  8.161673545837402,
  8.15420150756836,
  7.175506591796875,
  7.243679046630859,
  6.209869384765625,
  6.533681869506836,
  5.199947357177734]}

In [8]:
#On récupère le meilleur modèle, stocké sous ce nom par Flair
model = SequenceTagger.load('./best-model.pt')

2020-07-18 17:04:20,619 loading file ./best-model.pt


In [18]:
data_folder = "./flair"

columns = {0: 'text', 1: 'ner'}
corpus2: Corpus = ColumnCorpus(data_folder, columns,
                                    train_file=None,
                                      test_file='test.txt',
                                      )

2020-07-18 17:06:39,675 Reading data from flair
2020-07-18 17:06:39,676 Train: flair/train.txt
2020-07-18 17:06:39,677 Dev: None
2020-07-18 17:06:39,678 Test: flair/test.txt


In [13]:
result, eval_loss = model.evaluate(corpus2.test)

In [14]:
print(result.detailed_results)
    


Results:
- F-score (micro) 0.5556
- F-score (macro) 0.4444
- Accuracy 0.5556

By class:
                   precision    recall  f1-score   support

I-location_detail     1.0000    0.5000    0.6667         2
                O     0.5000    1.0000    0.6667         4
         I-arrive     1.0000    0.0000    0.0000         3

         accuracy                         0.5556         9
        macro avg     0.8333    0.5000    0.4444         9
     weighted avg     0.7778    0.5556    0.4444         9



In [15]:
print(corpus2.test[0].to_tagged_string('ner'))

philadelphia <I-location_detail> to dallas <I-location_detail> arriving before <I-arrive> 1 <I-arrive> in the afternoon <I-arrive>


In [16]:

test = corpus2.test[0]
sentence =test.to_plain_string()
print(sentence)
tests = Sentence(sentence)
model.predict(tests)
print(tests.to_tagged_string())

philadelphia to dallas arriving before 1 in the afternoon
philadelphia <I-location_detail> to dallas arriving before 1 in the afternoon
